In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import altair as alt

In [ ]:
india = gpd.read_file('https://github.com/fazilkhan/storytellingWithData/blob/main/INDIASHAPEFILE.zip?raw=true')

In [ ]:
PunjabHaryana = india[(india.nam == "PUNJAB") | (india.nam == "HARYANA")]

In [ ]:
PunjabHaryana = PunjabHaryana.rename(columns={
    "coc": "COUNTRY",
    "nam": "STATE",
    "laa": "DISTRICT"
})

In [ ]:
PunjabHaryana.head()

CSVs used for concatenation here were exported from clean excel files for each year. This was done in a separate notebook.

In [ ]:
csv_file_list = ["/Users/khanfazil/Desktop/master's_project/GW_Levels_Station_Punjab/stationsPunjab.csv",
                "/Users/khanfazil/Desktop/master's_project/GW_Levels_Station_Haryana/stationsHaryana.csv"]

list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename))

df = pd.concat(list_of_dataframes)

In [ ]:
df.head()

Removing null values

In [ ]:
df['MAY (Level m)'] = df['MAY (Level m)'] * 3.281
df = df.rename(columns={'MAY (Level m)': 'MAY (Level ft)'})

In [ ]:
dfstations18 = df[(df.YEAR == 2018) & (~df["MAY (Level ft)"].isna())]

In [ ]:
dfstations18.to_csv("PunjabHaryanaStationsMay18.csv", index=False)

In [ ]:
df.to_csv("PunjabHaryanaStations.csv", index=False)

In [ ]:
dfGeo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

In [ ]:
dfGeo18 = dfGeo[dfGeo.YEAR == 2018]

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
PunjabHaryanaMap = alt.Chart(PunjabHaryana).mark_geoshape(
    fill='whitesmoke',
    stroke='gray',
    strokeWidth=0.5
).encode(
    tooltip=['STATE','DISTRICT']
).properties(
    width=500,
    height=500,
    title='Punjab and Haryana Station-wise GW Levels, 2018'
)

map18 = alt.Chart(dfGeo18).mark_geoshape(
    size=5
).encode(
    color=alt.Color('MAY (Level ft):Q', title='MAY (Level m)', scale=alt.Scale(scheme='inferno', domain=[250,0])),
    tooltip=['MAY (Level ft):Q','STATE','DISTRICT','STATION', 'YEAR'],
).properties(
    title='Station-wise Groudnwater Depth 2018',
    projection={'type': 'mercator', 'pointRadius':3.5}
)

PunjabHaryanaMap + map18

In [ ]:
(PunjabHaryanaMap + map18).save('PunjHarStations18.html')

In [ ]:
# map18 = alt.Chart(dfGeo).mark_geoshape().encode(
#     color=alt.Color('MAY (Level m):Q', title='MAY (Level m)', scale=alt.Scale(scheme='inferno', domain=[55,0])),
#     tooltip=['MAY (Level m):Q','STATE','DISTRICT','STATION', 'YEAR'],
# ).properties(
#     title='Station-wise Groudnwater Depth 2018, Punjab and Haryana',
#     width=500,
#     height=500
# ).configure_view(
#     strokeWidth=0
# )

# map18